# **Practical Example: Integrating Data from Multiple Sources for COVID-19 Cases in the US**

In this exercise, we will walk through a practical example of data integration using the following sources:
1.	**Open Government API:** We’ll use the **USA Facts COVID-19 data API** to fetch COVID-19 case numbers for different states.
2.	**CSV File:** A local CSV file will contain state population data, which will be merged with the COVID-19 case numbers to calculate the number of cases per capita.

We’ll use **Pandas** to handle the CSV file, Requests to interact with the API, and **PyMongo** to store the resulting dataset in **MongoDB**.


**Pre-requisites:**

* Basic knowledge of Python.
* MongoDB Atlas account (or a local MongoDB instance).
* Install the required Python libraries

## **Step 1: Extracting Data from Multiple Sources**

**Fetching Data from an API:**

APIs are a common source of real-time data. For this example, we’ll fetch COVID-19 case data from the **USA Facts COVID-19 API**.
Here’s how to make an API call using Python’s requests library:


In [1]:
import requests
import pandas as pd

# Define the API endpoint for COVID-19 data by state
api_url = "https://api.covidtracking.com/v1/states/current.json"

# Make a GET request to the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to a Pandas DataFrame
    covid_data = pd.DataFrame(response.json())
    print(covid_data.head())
else:
    print(f"API request failed with status code {response.status_code}")


       date state  positive  probableCases   negative  pending  \
0  20210307    AK     56886            NaN        NaN      NaN   
1  20210307    AL    499819       107742.0  1931711.0      NaN   
2  20210307    AR    324818        69092.0  2480716.0      NaN   
3  20210307    AS         0            NaN     2140.0      NaN   
4  20210307    AZ    826454        56519.0  3073010.0      NaN   

  totalTestResultsSource  totalTestResults  hospitalizedCurrently  \
0        totalTestsViral           1731628                   33.0   
1  totalTestsPeopleViral           2323788                  494.0   
2        totalTestsViral           2736442                  335.0   
3        totalTestsViral              2140                    NaN   
4        totalTestsViral           7908105                  963.0   

   hospitalizedCumulative  ...  dataQualityGrade  deathIncrease  \
0                  1293.0  ...              None              0   
1                 45976.0  ...              None      

In this code:

•	We use *requests.get()* to fetch data from the API.

•	If the API call is successful *(status code 200)*, the JSON response is converted to a Pandas DataFrame for further manipulation.

The resulting DataFrame contains COVID-19 case numbers by state, including fields such as state, positive (number of positive cases), death (number of deaths), and more.


**Reading Data from a CSV File**

Now, let’s read the population data from a CSV file. This data will later be merged with the COVID-19 case data to calculate cases per capita. Note that you have to add the CSV file "state_population.csv" to your google colab environment before executing the following code.

Here’s how to read a CSV file using **Pandas**:


In [3]:
# Load the state population data from a CSV file
population_data = pd.read_csv('./sample_data/state_population.csv')

# Display the first few rows of the DataFrame
print(population_data.head())


  state  population
0    CA    39538223
1    TX    29145505
2    FL    21538187
3    NY    20201249
4    PA    13002700


## **Step 2: Data Transformation**

Once we’ve extracted the data from both sources (API and CSV), we need to transform it. The key transformation here is merging the datasets and calculating cases per capita.


**Merging Data from Different Sources**

To merge the COVID-19 case data with the population data, we use the common key state. We’ll use Pandas' merge() function for this.


In [4]:
# Merge the COVID-19 case data with the population data
merged_data = pd.merge(covid_data, population_data, left_on='state', right_on='state')

# Display the merged data
print(merged_data.head())


       date state  positive  probableCases   negative  pending  \
0  20210307    CA   3501394            NaN        NaN      NaN   
1  20210307    FL   1909209       360372.0  9339038.0   8544.0   
2  20210307    NY   1681169            NaN        NaN      NaN   
3  20210307    PA    948643       134103.0  3942124.0      NaN   
4  20210307    TX   2686818       365961.0        NaN      NaN   

     totalTestResultsSource  totalTestResults  hospitalizedCurrently  \
0           totalTestsViral          49646014                 4291.0   
1  totalTestEncountersViral          22339182                 3307.0   
2  totalTestEncountersViral          39695100                 4789.0   
3  totalTestEncountersViral          10681716                 1587.0   
4           totalTestsViral          19907384                 4721.0   

   hospitalizedCumulative  ...  deathIncrease  hospitalizedIncrease  \
0                     NaN  ...            258                     0   
1                 82237.0  .

This will create a DataFrame that contains both COVID-19 case data and population for each state.

**Calculating Cases Per Capita**

Now that the data is merged, we can perform a transformation to calculate the number of cases per capita (cases per 100,000 people).


In [5]:
# Calculate cases per capita (cases per 100,000 people)
merged_data['cases_per_capita'] = (merged_data['positive'] / merged_data['population']) * 100000

# Display the updated DataFrame
print(merged_data[['state', 'positive', 'population', 'cases_per_capita']].head())

  state  positive  population  cases_per_capita
0    CA   3501394    39538223       8855.719186
1    FL   1909209    21538187       8864.297631
2    NY   1681169    20201249       8322.104242
3    PA    948643    13002700       7295.738577
4    TX   2686818    29145505       9218.635944


This transformation adds a new column cases_per_capita that gives a normalized view of the number of COVID-19 cases relative to the state’s population.


**Loading the Integrated Data into MongoDB**

Finally, load the integrated dataset into a MongoDB collection for storage.

Ensure you have MongoDB running locally or use MongoDB Atlas. Connect to MongoDB using PyMongo.


In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['MongoDB_db']

# Insert the integrated data into MongoDB
db.covid_data.insert_many(merged_data.to_dict('records'))

print("Data loaded into MongoDB successfully!")


# **Congratulations on completing this practical exercise!**